# **Create Model**

Train and validation the model using `wider_face_train` and `wider_face_val` dataset.

### Import and Define Directory

In [1]:
import os
import sys
import numpy as np
import skimage.io
import skimage.draw
import json
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from pycocotools.coco import COCO
import pycocotools.mask as maskUtils

# Root directory of the project
ROOT_DIR = os.path.abspath('D:/Hanifan/Face-Detection-MaskRCNN')

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import utils
from mrcnn.utils import Dataset

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

### Setup Configuration

In [2]:
class DatasetConfig(Config):
    GPU_COUNT = 1
    NAME = "Face_Detection"
    IMAGE_MIN_DIM = 800
    IMAGE_MAX_DIM = 1024
    IMAGES_PER_GPU = 1 
    NUM_CLASSES = 1 + 1  # background + face
    STEPS_PER_EPOCH = 100

### Define Dataset

In [3]:
class FaceDataset(Dataset):
    def load_face(self, dataset_dir):
        # Add classes
        self.add_class("face", 1, "face")
        
        annotations_dir = os.path.join(dataset_dir, 'annotations')
        images_dir = os.path.join(dataset_dir, 'images')
        
        for json_file in os.listdir(annotations_dir):
            if json_file.endswith(".json"):
                json_path = os.path.join(annotations_dir, json_file)
                
                with open(json_path, 'r') as jsonfile:
                    boundingboxes = json.load(jsonfile)
                
                image_filename = json_file[:-5] + ".jpg"
                
                image_path = None
                for subdir, _, files in os.walk(images_dir):
                    if image_filename in files:
                        image_path = os.path.join(subdir, image_filename)
                        break
                if not image_path:
                    print(f"Image file does not exist: {image_filename}")
                    continue
                
                image = skimage.io.imread(image_path)
                if image.shape[0] > 1024:
                    continue
                
                self.add_image(
                    "face",
                    image_id=image_filename,  # Use filename as a unique id
                    path=image_path,
                    width=image.shape[1],
                    height=image.shape[0],
                    boundingbox=boundingboxes
                )

    def load_mask(self, image_id):
        """
        Generate instance masks for shapes of given image ID
        """
        image_info = self.image_info[image_id]
        if image_info["source"] != "face":
            return super(self.__class__, self).load_mask(image_id)
        
        info = self.image_info[image_id]
        boundingboxes = info['boundingbox']

        # # Print bounding boxes for debugging
        # print(f"Bounding boxes for image {image_id}: {boundingboxes}")

        # Initialize mask
        mask = np.zeros([info['height'], info['width'], len(boundingboxes)], dtype=np.uint8)
        
        for i, key in enumerate(boundingboxes.keys()):
            box = boundingboxes[key]
            if 'x' in box and 'y' in box:
                x = np.clip(box['x'], 0, info['width'] - 1)
                y = np.clip(box['y'], 0, info['height'] - 1)
                rr, cc = skimage.draw.polygon(y, x)
                mask[rr, cc, i] = 1
            else:
                print(f"Invalid bounding box format for image {image_id}: {box}")
        
        return mask, np.ones([mask.shape[-1]], dtype=np.int32)
    
    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "face":
            return info["path"]
        else:
            return super(self.__class__, self).image_reference(image_id)

### Train or Inference the Model

In [4]:
# Choose 'train' or 'inference'
if __name__ == "__main__":
    command = "train"

    if command == "train":
        config = DatasetConfig()
    else:
        class InferenceConfig(DatasetConfig):
            GPU_COUNT = 1
            IMAGES_PER_GPU = 1
        config = InferenceConfig()
    config.display()

    if command == "train":
        # Create model in training mode
        model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)
        
        # Load pre-trained weights (optional, if available)
        coco_weights_path = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
        if not os.path.exists(coco_weights_path):
            utils.download_trained_weights(coco_weights_path)

        # Load weights trained on MS-COCO
        model.load_weights(coco_weights_path, by_name=True,
                        exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                    "mrcnn_bbox", "mrcnn_mask"])
        # Training dataset
        print("Preparing training dataset...")
        dataset_train = FaceDataset()
        dataset_train.load_face("D:/Hanifan/Face-Detection-MaskRCNN/wider_face_split/wider_face_train")
        dataset_train.prepare()

        # Validation dataset
        print("Preparing validation dataset...")
        dataset_val = FaceDataset()
        dataset_val.load_face("D:/Hanifan/Face-Detection-MaskRCNN/wider_face_split/wider_face_val")
        dataset_val.prepare()

        # Proceed with training if all files are validated
        print("Starting training...")
        history = model.train(dataset_train, dataset_val,
                                learning_rate=config.LEARNING_RATE / 10,
                                epochs=30,
                                layers='heads')
        print(history)
    else:
        model = modellib.MaskRCNN(mode="inference", config=config, model_dir=MODEL_DIR)


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

c:\Users\User\miniconda3\envs\anis\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/30


c:\Users\User\miniconda3\envs\anis\lib\site-packages\tensorflow\python\framework\indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/Shape:0", shape=(4,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
c:\Users\User\miniconda3\envs\anis\lib\site-packages\tensorflow\python\framework\indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub_1:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_gr

100/100 [==============================] - ETA: 0s - batch: 49.5000 - size: 1.0000 - loss: 2.9028 - rpn_class_loss: 0.2087 - rpn_bbox_loss: 1.3977 - mrcnn_class_loss: 0.1698 - mrcnn_bbox_loss: 0.6472 - mrcnn_mask_loss: 0.4794

c:\Users\User\miniconda3\envs\anis\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


100/100 [==============================] - 162s 1s/step - batch: 49.5000 - size: 1.0000 - loss: 2.9028 - rpn_class_loss: 0.2087 - rpn_bbox_loss: 1.3977 - mrcnn_class_loss: 0.1698 - mrcnn_bbox_loss: 0.6472 - mrcnn_mask_loss: 0.4794 - val_loss: 2.1130 - val_rpn_class_loss: 0.1063 - val_rpn_bbox_loss: 0.8590 - val_mrcnn_class_loss: 0.1882 - val_mrcnn_bbox_loss: 0.5953 - val_mrcnn_mask_loss: 0.3642
Epoch 2/30
100/100 [==============================] - 180s 2s/step - batch: 49.5000 - size: 1.0000 - loss: 2.0502 - rpn_class_loss: 0.1190 - rpn_bbox_loss: 0.8092 - mrcnn_class_loss: 0.2058 - mrcnn_bbox_loss: 0.5191 - mrcnn_mask_loss: 0.3971 - val_loss: 1.9885 - val_rpn_class_loss: 0.1656 - val_rpn_bbox_loss: 0.6792 - val_mrcnn_class_loss: 0.2056 - val_mrcnn_bbox_loss: 0.5268 - val_mrcnn_mask_loss: 0.4113
Epoch 3/30
100/100 [==============================] - 143s 1s/step - batch: 49.5000 - size: 1.0000 - loss: 1.8438 - rpn_class_loss: 0.1232 - rpn_bbox_loss: 0.6373 - mrcnn_class_loss: 0.1883 - m

### Convert the Model

In [8]:
model_path = os.path.join(MODEL_DIR, "mask_rcnn_face.h5")
model.keras_model.save_weights(model_path)